In [38]:
import ee
import geemap
import logging
import multiprocessing
import os
import requests
import shutil
from retry import retry

In [39]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AbUR2VNLIW-MxYu0psvOldxjrfByqfdJzm_3H2sVzfyPW0XS6VXuOr21Y_M

Successfully saved authorization token.


In [40]:
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [41]:
Map = geemap.Map()

In [42]:
ds = ee.FeatureCollection("projects/ee-pnolaraj/assets/Ktm_Shp")
Map.addLayer(ds, {}, "Kathmandu")

In [43]:
# Provide any string on the name for filtering
def School_CoOrdinates(String):
    filtered_features = ds.filter(ee.Filter.stringContains('name', String))

    # Map over the filtered features to extract their coordinates
    coordinates_list = filtered_features.geometry().coordinates()
    
    #Places and Co ordinates values    
    place_names = filtered_features.aggregate_array('name').getInfo()
    CoOrdinatesList = coordinates_list.getInfo()
   
    
    #Sorting the boundary Cordinates to point
    CoOrdinates_sorted = []
    for i in CoOrdinatesList:
        for j in i:
            for k in j:
                CoOrdinates_sorted.append([k[0], k[1]])
                break
            break
            
            
    return [CoOrdinates_sorted, place_names]

In [44]:
String = "School"

Result = School_CoOrdinates(String)
CoOrdinate, Place_Name = Result[0], Result[1]

In [46]:
print(len(Place_Name))
print(CoOrdinate)
print(Place_Name)

950
[[85.313269329776, 27.70610267170353], [85.31498610650488, 27.734284206827198], [85.29423342301065, 27.693358567401674], [85.28292959018997, 27.721740757761722], [85.37005594240519, 27.729508489209426], [85.35136343486286, 27.72786307837473], [85.28267543470776, 27.684511728840924], [85.33910086892399, 27.741053139861666], [85.29835359443926, 27.72699357409864], [85.3463424368404, 27.718235928439515], [85.35892157302118, 27.725245586301796], [85.33639864509503, 27.697202348059182], [85.34392117142062, 27.685737966411104], [85.29453218985925, 27.677542124039814], [85.38263511697676, 27.729205262088986], [85.30140811157702, 27.700876654269454], [85.2787024050865, 27.68339246369056], [85.30372238006285, 27.71881111796621], [85.30396318271198, 27.698834340873695], [85.35675444220976, 27.68539464748273], [85.34715399515524, 27.700078394610653], [85.28699631221346, 27.69930252736018], [85.29580299608322, 27.70069380892194], [85.35075248870602, 27.732549636706654], [85.33560940016929, 27.

In [47]:
params = {
    'count': 100,  # How many image chips to export
    'buffer': 100,  # The buffer distance (m) around each point
    'scale': 100,  # The scale to do stratified sampling
    'seed': 1,  # A randomization seed to use for subsampling.
    'dimensions': '256x256',  # The dimension of each image chip
    'format': "geojson",  # The output image format, can be png, jpg, ZIPPED_GEO_TIFF, GEO_TIFF, NPY
    'prefix': f'{String}_',  # The filename prefix
    'processes': 25,  # How many processes to used for parallel processing
    'out_dir': 'Output',  # The output directory. Default to the current working directly
}

In [48]:
image = ee.FeatureCollection("GOOGLE/Research/open-buildings/v2/polygons")

In [49]:
def GEOJSON_from_Point(Point, Name):
    image = ee.FeatureCollection("GOOGLE/Research/open-buildings/v2/polygons")
    point_geom = ee.Geometry.Point(Point[0], Point[1])
    buffer = point_geom.buffer(buffer_radius)
    clipped_collection = image.filterBounds(buffer)
    download_url = clipped_collection.getDownloadURL(filetype='geojson')
    
    r = requests.get(download_url, stream=True)
    if r.status_code != 200:
        r.raise_for_status()
        
        
    ext = params['format']
    out_dir = os.path.abspath(params['out_dir'])    
    basename = str(Name)
    replacements = [",", "/", "\\", "?", "*", ":", "<", ">", "|", '"']
    for character in replacements:
        if character in basename:
                    basename = basename.replace(character, " ")  
    filename = f"{out_dir}/{params['prefix']}{basename}.{ext}"
                    
                    
    with open(filename, 'wb') as out_file:
        shutil.copyfileobj(r.raw, out_file)
    print("Done: ", basename)
    

In [ ]:
buffer_radius = 30
i=1
for Point, Name in zip(CoOrdinate, Place_Name):
    if Name =="Manakamana English Boarding School":
        i = 2
        print(Name)
    if i == 2:
        GEOJSON_from_Point(Point, Name)

Manakamana English Boarding School
Done:  Manakamana English Boarding School
Done:  New Tulips Secondary School
Done:  Hebron Public School
Done:  Subash Smrity School
Done:  Prabhat Secondary School
Done:  Kathmandu Xavier Public School
Done:  Shri Pashupati Mitra Secondary School
Done:  Arunima Higher Secondary School
Done:  Gurukul National School
Done:  Prerana School
Done:  Prerana School
Done:  Supreme International High School
Done:  Ananda Bhumi High School
Done:  Deerwalk Sifal School
Done:  Srijana Vidhya Mandir Secondary School
Done:  Mandala International School
Done:  Swornim International Model School
Done:  City English School
Done:  Clinton School
Done:  Charumati English School
Done:  English Preparatory School
Done:  Nirmal Vidyapeeth Secondary School
Done:  Annapurna Higher Secondary School
Done:  Nepal Yubak Secondary School
Done:  Surena English Boarding School
Done:  Pennwood School
Done:  Columbus International School
Done:  Kathmandu Barsha School
Done:  Shri Gy

Done:  Mega International School
Done:  National Orchid Boarding School
Done:  Children Herald English School
Done:  The White Mountain School
Done:  The Skylark English School
Done:  Jalupa Secondary School
Done:  Kathmandu Vidya Kunja School
Done:  Abhiyan Public School
Done:  Bright Heaven Boarding School
Done:  Shree Narayan Primary School
Done:  Sankhu Palubari Community School
Done:  Mount Summit Boarding School
Done:  Children's Garden Pre-School
Done:  Mangal Higher Secondary School
Done:  Canvas English Boarding School
Done:  Riviera International School
Done:  Kamana HS School
Done:  Shree Nepal Rastriya Nirman Higher Secondary School
Done:  Tej Binayak School
Done:  Megus English School
Done:  Kirtipur Secondary School
Done:  Chalnakhel Lower Secondary School
Done:  Laboratory Higher SecondarySchool
Done:  Summer Hill School
Done:  Loyala Higher Secondary School
Done:  Tyauda Secondary School
Done:  Hill Town Higher Secondary School
Done:  Pragyan School
Done:  Pancha Kanya 

Done:  Rudramati Primary School
Done:  Shree Taleju Bhawani Primary School
Done:  Khagendra New Life Special Education Secondary School
Done:  Columbus School
Done:  Gitamata Higher Secondary School
Done:  Krishna Mandir Primary School
Done:  Nepal Rastriya Lower Secondary School
Done:  Shree Ganesh Primary School
Done:  Champak Binayak Primary School
Done:  Shree Kanti Bhairab Gurukul School
Done:  Kesh Chandra Primary School
Done:  Ujjwal School Office Building
Done:  Mangladevi Secondary School
Done:  Himalayan Internatinal Model High School


In [ ]:
out_dir = os.path.abspath(params['out_dir'])  
TextFile = f"{out_dir}/{params['prefix']}CoOrdinates.txt"
with open(TextFile, 'w') as f:
    for Point, Name in zip(CoOrdinate, Place_Name):
        f.write(str(Point[0]))
        f.write('\t')        
        f.write(str(Point[1]))
        f.write('\t')
        f.write(Name)
        f.write('\n')